In [86]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from tensorflow.keras import regularizers
from tensorflow.keras import activations


In [87]:
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)
print(device_lib.list_local_devices())


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1602041109249738594
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9066120598516028213
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12667430510003122803
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7009469728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4422863654415315339
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, com

In [88]:
directory = "./data/eeg_feature_smooth/1/"
directories = ["./data/eeg_feature_smooth/{}/".format(i+1) for i in range(3)] 
print(directories)
channel_coords = [['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'], ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'], ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'], ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'], ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'], ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'], ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'], ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']]

channel_list = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']
print(len(channel_coords), len(channel_coords[0]))
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if (channel_list[n] == x):
                coord_dict[n] = (i,j)
print(coord_dict)

['./data/eeg_feature_smooth/1/', './data/eeg_feature_smooth/2/', './data/eeg_feature_smooth/3/']
8 9
{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}


In [89]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64)) # features = 4 datatypes*(8 x 9 eeg channel locs)*5 frequency bands*64 timestamps(zero padded) // trials = (3 sessions) x 15 people x 24 labels 
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                for l, channel in enumerate(sample):
                    array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

print(array.shape)



./data/eeg_feature_smooth/1/
./data/eeg_feature_smooth/2/
./data/eeg_feature_smooth/3/
(3, 15, 24, 4, 8, 9, 5, 64)


In [90]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)
X_loso = array[:,:,:,1,:,:,]
X_loso = np.transpose(X_loso, (0,1,2,6,3,4,5))
print(X_loso.shape)



(1080, 4, 8, 9, 5, 64)
(3, 15, 24, 64, 8, 9, 5)


In [91]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)

print(y.shape)
y_loso = np.reshape(y, (3,15,24))
print(y_loso.shape)


(1080,)
(3, 15, 24)


In [104]:
def crossval_loso(generate_model, n_epochs, X, y):
    cvscores = []
    for i in range(15):
        a = [x for x in range(15) if x != i]
        print(a)
        X_train = X[:,a,:,:,:,:,:]
        X_test =  X[:,[i],:,:,:,:,:]
        X_train = X_train.reshape(np.prod(X_train.shape[0:3]), *X_train.shape[3:])
        X_test = X_test.reshape(np.prod(X_test.shape[0:3]), *X_test.shape[3:])
        y_train = y[:,a,:]
        y_test = y[:, [i], :]
        y_train = y_train.flatten()
        y_test = y_test.flatten()
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        model = generate_model()
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        print('------------------------------------------------------------------------')
        print(f'Training for fold {i} ...')
        model.fit(X_train, y_train, epochs=n_epochs, verbose=1) # validation_split=0.2)
        scores = model.evaluate(X_test, y_test, verbose=2)
        print("Score for fold %d - %s: %.6f%%" % (i, model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1])
    print('------------------------------------------------------------------------')
    print("Avg accuracies: %.6f%% (+/- %.6f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [7]:
def crossval(generate_model, n_epochs, X_train, y_train, X_test, y_test, filename = None):
    kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
    bestmodel = None
    bestAcc = 0
    cvscores = []
    fold = 1
    for train, test in kfold.split(X_train, y_train):
        model = generate_model()
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold} ...')
        model.fit(X_train[train], y_train[train],epochs=n_epochs, verbose=1) # validation_split=0.2)
        scores = model.evaluate(X_train[test], y_train[test], verbose=1)
        print("Score for fold %d - %s: %.2f%%" % (fold, model.metrics_names[1], scores[1]*100))
        if(scores[1] > bestAcc):
            bestAcc = scores[1]
            bestmodel = model
        cvscores.append(scores[1] * 100)
        fold += 1
    print('------------------------------------------------------------------------')
    print("Avg accuracies: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    test_loss, test_acc = bestmodel.evaluate(X_test,  y_test, verbose=2)
    print('\nTest accuracy:', test_acc)
    if filename:
        pickle.dump( bestmodel, open( filename, "wb" ) )

In [8]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [9]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(
32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [10]:
def LSTM():
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
    ])

In [11]:
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [12]:
def twodConv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv2D(
32, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Conv2D(
64, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [94]:
def threedConv():
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv3D(32, kernel_size=5, padding="same", input_shape=X.shape[1:]))
    model.add(tf.keras.layers.BatchNormalization(trainable=False))
    model.add(tf.keras.layers.Activation(activations.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization(trainable=False))
    model.add(tf.keras.layers.Activation(activations.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    return model

In [24]:
def convlstm2d():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(16, kernel_size=5, padding="same", return_sequences = False, input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(10))
    return model

In [14]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [15]:
crossval(dense, 40, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/40
21/21 [==============================] - 0s 11ms/step - loss: 705096512.0000 - accuracy: 0.2978
Epoch 2/40
21/21 [==============================] - 0s 11ms/step - loss: 598244352.0000 - accuracy: 0.3796
Epoch 3/40
21/21 [==============================] - 0s 11ms/step - loss: 122490768.0000 - accuracy: 0.3904
Epoch 4/40
21/21 [==============================] - 0s 11ms/step - loss: 40040048.0000 - accuracy: 0.4228
Epoch 5/40
21/21 [==============================] - 0s 10ms/step - loss: 117029792.0000 - accuracy: 0.4707
Epoch 6/40
21/21 [==============================] - 0s 10ms/step - loss: 39676196.0000 - accuracy: 0.4660
Epoch 7/40
21/21 [==============================] - 0s 10ms/step - loss: 57045640.0000 - accuracy: 0.3966
Epoch 8/40
21/21 [==============================] - 0s 11ms/step - loss: 1703190.8750 - accuracy: 0.4105
Epoch 9/40
21/21 [==============================] - 

21/21 [==============================] - 0s 10ms/step - loss: 50616.5703 - accuracy: 0.8596
Epoch 37/40
21/21 [==============================] - 0s 10ms/step - loss: 17415.6348 - accuracy: 0.9259
Epoch 38/40
21/21 [==============================] - 0s 11ms/step - loss: 25658.0723 - accuracy: 0.9213
Epoch 39/40
21/21 [==============================] - 0s 11ms/step - loss: 16129.5801 - accuracy: 0.9367
Epoch 40/40
7/7 [==============================] - 0s 6ms/step - loss: 2225948416.0000 - accuracy: 0.6481
Score for fold 2 - accuracy: 64.81%
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/40
21/21 [==============================] - 0s 11ms/step - loss: 455926496.0000 - accuracy: 0.3025
Epoch 2/40
21/21 [==============================] - 0s 11ms/step - loss: 60868176.0000 - accuracy: 0.3164
Epoch 3/40
21/21 [==============================] - 0s 11ms/step - loss: 18653958.0000 - accuracy: 0.3534
Epoch 4/40
21/21 [====================

Epoch 30/40
21/21 [==============================] - 0s 10ms/step - loss: 188406.0312 - accuracy: 0.7978
Epoch 31/40
21/21 [==============================] - 0s 10ms/step - loss: 140179.1406 - accuracy: 0.8117
Epoch 32/40
21/21 [==============================] - 0s 10ms/step - loss: 110161.2109 - accuracy: 0.8040
Epoch 33/40
21/21 [==============================] - 0s 10ms/step - loss: 71950.0938 - accuracy: 0.8503
Epoch 34/40
21/21 [==============================] - 0s 10ms/step - loss: 60438.2148 - accuracy: 0.8704
Epoch 35/40
21/21 [==============================] - 0s 10ms/step - loss: 43293.7188 - accuracy: 0.8781
Epoch 36/40
21/21 [==============================] - 0s 10ms/step - loss: 37403.8086 - accuracy: 0.8889
Epoch 37/40
21/21 [==============================] - 0s 10ms/step - loss: 43030.2188 - accuracy: 0.8904
Epoch 38/40
21/21 [==============================] - 0s 11ms/step - loss: 40405.0469 - accuracy: 0.8904
Epoch 39/40
21/21 [==============================] - 0s 10ms/

In [16]:
crossval(single_conv, 40, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/40
21/21 [==============================] - 0s 13ms/step - loss: 136558736.0000 - accuracy: 0.2917
Epoch 2/40
21/21 [==============================] - 0s 11ms/step - loss: 18294216.0000 - accuracy: 0.2809
Epoch 3/40
21/21 [==============================] - 0s 11ms/step - loss: 11852860.0000 - accuracy: 0.3194
Epoch 4/40
21/21 [==============================] - 0s 10ms/step - loss: 29593734.0000 - accuracy: 0.3750
Epoch 5/40
21/21 [==============================] - 0s 11ms/step - loss: 4453705.5000 - accuracy: 0.3040
Epoch 6/40
21/21 [==============================] - 0s 10ms/step - loss: 4178130.0000 - accuracy: 0.3935
Epoch 7/40
21/21 [==============================] - 0s 11ms/step - loss: 1451300.3750 - accuracy: 0.4321
Epoch 8/40
21/21 [==============================] - 0s 10ms/step - loss: 15595266.0000 - accuracy: 0.4506
Epoch 9/40
21/21 [==============================] - 0s 10

21/21 [==============================] - 0s 10ms/step - loss: 21198.2266 - accuracy: 0.4985
Epoch 37/40
21/21 [==============================] - 0s 10ms/step - loss: 15019.5479 - accuracy: 0.4753
Epoch 38/40
21/21 [==============================] - 0s 10ms/step - loss: 12194.0918 - accuracy: 0.4753
Epoch 39/40
21/21 [==============================] - 0s 10ms/step - loss: 10184.9609 - accuracy: 0.5509
Epoch 40/40
7/7 [==============================] - 0s 6ms/step - loss: 145674016.0000 - accuracy: 0.4676
Score for fold 2 - accuracy: 46.76%
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/40
21/21 [==============================] - 0s 11ms/step - loss: 92549592.0000 - accuracy: 0.3102
Epoch 2/40
21/21 [==============================] - 0s 11ms/step - loss: 37477856.0000 - accuracy: 0.3735
Epoch 3/40
21/21 [==============================] - 0s 10ms/step - loss: 39352720.0000 - accuracy: 0.3580
Epoch 4/40
21/21 [======================

21/21 [==============================] - 0s 11ms/step - loss: 57609.7383 - accuracy: 0.6343
Epoch 30/40
21/21 [==============================] - 0s 10ms/step - loss: 52260.8516 - accuracy: 0.6327
Epoch 31/40
21/21 [==============================] - 0s 10ms/step - loss: 49104.5234 - accuracy: 0.6373
Epoch 32/40
21/21 [==============================] - 0s 10ms/step - loss: 44774.2305 - accuracy: 0.6451
Epoch 33/40
21/21 [==============================] - 0s 10ms/step - loss: 41508.9102 - accuracy: 0.6574
Epoch 34/40
21/21 [==============================] - 0s 10ms/step - loss: 40346.5977 - accuracy: 0.6590
Epoch 35/40
21/21 [==============================] - 0s 10ms/step - loss: 37114.0312 - accuracy: 0.6775
Epoch 36/40
21/21 [==============================] - 0s 10ms/step - loss: 39630.6250 - accuracy: 0.6867
Epoch 37/40
21/21 [==============================] - 0s 10ms/step - loss: 33239.0938 - accuracy: 0.6883
Epoch 38/40
21/21 [==============================] - 0s 10ms/step - loss: 29

In [15]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,:]

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 20)


In [18]:
crossval(threedConv, 100, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
21/21 [==============================] - 1s 25ms/step - loss: 1131064704.0000 - accuracy: 0.2778
Epoch 2/100
21/21 [==============================] - 0s 23ms/step - loss: 292685888.0000 - accuracy: 0.3102
Epoch 3/100
21/21 [==============================] - 0s 22ms/step - loss: 177587392.0000 - accuracy: 0.3765
Epoch 4/100
21/21 [==============================] - 0s 22ms/step - loss: 262410560.0000 - accuracy: 0.4506
Epoch 5/100
21/21 [==============================] - 0s 22ms/step - loss: 37507780.0000 - accuracy: 0.5139
Epoch 6/100
21/21 [==============================] - 0s 22ms/step - loss: 3107889.2500 - accuracy: 0.5401
Epoch 7/100
21/21 [==============================] - 0s 22ms/step - loss: 93415408.0000 - accuracy: 0.5432
Epoch 8/100
21/21 [==============================] - 0s 22ms/step - loss: 134789872.0000 - accuracy: 0.5216
Epoch 9/100
21/21 [=======================

21/21 [==============================] - 0s 22ms/step - loss: 892625216.0000 - accuracy: 0.5108
Epoch 78/100
21/21 [==============================] - 0s 22ms/step - loss: 30286020.0000 - accuracy: 0.6636
Epoch 79/100
21/21 [==============================] - 0s 22ms/step - loss: 46501856.0000 - accuracy: 0.7160
Epoch 80/100
21/21 [==============================] - 0s 22ms/step - loss: 49356596.0000 - accuracy: 0.7068
Epoch 81/100
21/21 [==============================] - 0s 22ms/step - loss: 2894925.2500 - accuracy: 0.7022
Epoch 82/100
21/21 [==============================] - 0s 22ms/step - loss: 350551936.0000 - accuracy: 0.6651
Epoch 83/100
21/21 [==============================] - 0s 22ms/step - loss: 36100596.0000 - accuracy: 0.6914
Epoch 84/100
21/21 [==============================] - 0s 22ms/step - loss: 522695840.0000 - accuracy: 0.6883
Epoch 85/100
21/21 [==============================] - 0s 22ms/step - loss: 509402368.0000 - accuracy: 0.7099
Epoch 86/100
21/21 [==================

21/21 [==============================] - 0s 23ms/step - loss: 498228.5312 - accuracy: 0.7593
Epoch 52/100
21/21 [==============================] - 0s 23ms/step - loss: 637123.9375 - accuracy: 0.7361
Epoch 53/100
21/21 [==============================] - 0s 22ms/step - loss: 3378675.7500 - accuracy: 0.7701
Epoch 54/100
21/21 [==============================] - 0s 23ms/step - loss: 551544.3750 - accuracy: 0.7731
Epoch 55/100
21/21 [==============================] - 0s 23ms/step - loss: 1331601.6250 - accuracy: 0.7639
Epoch 56/100
21/21 [==============================] - 0s 22ms/step - loss: 1092069.0000 - accuracy: 0.7809
Epoch 57/100
21/21 [==============================] - 0s 23ms/step - loss: 126928.3672 - accuracy: 0.7577
Epoch 58/100
21/21 [==============================] - 0s 22ms/step - loss: 334891.4062 - accuracy: 0.7840
Epoch 59/100
21/21 [==============================] - 0s 22ms/step - loss: 2787866.0000 - accuracy: 0.7685
Epoch 60/100
21/21 [==============================] - 0

21/21 [==============================] - 0s 22ms/step - loss: 1093368.6250 - accuracy: 0.5108
Epoch 26/100
21/21 [==============================] - 0s 23ms/step - loss: 821669.6250 - accuracy: 0.5293
Epoch 27/100
21/21 [==============================] - 0s 23ms/step - loss: 20557882.0000 - accuracy: 0.5108
Epoch 28/100
21/21 [==============================] - 0s 22ms/step - loss: 239758.9688 - accuracy: 0.5787
Epoch 29/100
21/21 [==============================] - 0s 23ms/step - loss: 209459.6875 - accuracy: 0.5478
Epoch 30/100
21/21 [==============================] - 0s 22ms/step - loss: 8004561.5000 - accuracy: 0.5478
Epoch 31/100
21/21 [==============================] - 0s 22ms/step - loss: 95219.0156 - accuracy: 0.5694
Epoch 32/100
21/21 [==============================] - 0s 22ms/step - loss: 161784.2812 - accuracy: 0.5679
Epoch 33/100
21/21 [==============================] - 0s 23ms/step - loss: 138677.6406 - accuracy: 0.5417
Epoch 34/100
21/21 [==============================] - 0s

Epoch 1/100
21/21 [==============================] - 1s 25ms/step - loss: 1431995008.0000 - accuracy: 0.2809
Epoch 2/100
21/21 [==============================] - 1s 25ms/step - loss: 818986304.0000 - accuracy: 0.3117
Epoch 3/100
21/21 [==============================] - 0s 22ms/step - loss: 37862832.0000 - accuracy: 0.4660
Epoch 4/100
21/21 [==============================] - 0s 22ms/step - loss: 42126552.0000 - accuracy: 0.4969
Epoch 5/100
21/21 [==============================] - 0s 22ms/step - loss: 14599386.0000 - accuracy: 0.5509
Epoch 6/100
21/21 [==============================] - 0s 22ms/step - loss: 15471282.0000 - accuracy: 0.5664
Epoch 7/100
21/21 [==============================] - 0s 22ms/step - loss: 5484245.5000 - accuracy: 0.5772
Epoch 8/100
21/21 [==============================] - 0s 22ms/step - loss: 12153482.0000 - accuracy: 0.5880
Epoch 9/100
21/21 [==============================] - 0s 23ms/step - loss: 128223.8906 - accuracy: 0.6188
Epoch 10/100
21/21 [=================

21/21 [==============================] - 0s 23ms/step - loss: 82177.2891 - accuracy: 0.7870
Epoch 78/100
21/21 [==============================] - 0s 23ms/step - loss: 211157.5469 - accuracy: 0.8040
Epoch 79/100
21/21 [==============================] - 0s 23ms/step - loss: 113229.0469 - accuracy: 0.8164
Epoch 80/100
21/21 [==============================] - 0s 22ms/step - loss: 77357.6875 - accuracy: 0.8086
Epoch 81/100
21/21 [==============================] - 0s 23ms/step - loss: 67699.8594 - accuracy: 0.8318
Epoch 82/100
21/21 [==============================] - 0s 22ms/step - loss: 124891.9062 - accuracy: 0.8071
Epoch 83/100
21/21 [==============================] - 0s 22ms/step - loss: 47398.2695 - accuracy: 0.8194
Epoch 84/100
21/21 [==============================] - 0s 22ms/step - loss: 2567550.0000 - accuracy: 0.8287
Epoch 85/100
21/21 [==============================] - 0s 22ms/step - loss: 40682.1992 - accuracy: 0.8611
Epoch 86/100
21/21 [==============================] - 0s 22ms/s

In [21]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,1]

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 5)


In [25]:
crossval(convlstm2d, 30, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/30
21/21 [==============================] - 29s 1s/step - loss: 1.5657 - accuracy: 0.3102
Epoch 2/30
21/21 [==============================] - 28s 1s/step - loss: 1.3098 - accuracy: 0.3781
Epoch 3/30
21/21 [==============================] - 29s 1s/step - loss: 1.2951 - accuracy: 0.4043
Epoch 4/30
21/21 [==============================] - 28s 1s/step - loss: 1.2192 - accuracy: 0.4244
Epoch 5/30
21/21 [==============================] - 29s 1s/step - loss: 1.1804 - accuracy: 0.4414
Epoch 6/30
21/21 [==============================] - 28s 1s/step - loss: 1.1614 - accuracy: 0.4738
Epoch 7/30
21/21 [==============================] - 28s 1s/step - loss: 1.1171 - accuracy: 0.4799
Epoch 8/30
21/21 [==============================] - 29s 1s/step - loss: 1.0225 - accuracy: 0.5139
Epoch 9/30
21/21 [==============================] - 29s 1s/step - loss: 1.0825 - accuracy: 0.4660
Epoch 10/30
21/21 [==

21/21 [==============================] - 29s 1s/step - loss: 0.8582 - accuracy: 0.5741
Epoch 19/30
21/21 [==============================] - 29s 1s/step - loss: 0.8555 - accuracy: 0.5833
Epoch 20/30
21/21 [==============================] - 29s 1s/step - loss: 0.9188 - accuracy: 0.5340
Epoch 21/30
21/21 [==============================] - 29s 1s/step - loss: 0.8492 - accuracy: 0.5910
Epoch 22/30
21/21 [==============================] - 29s 1s/step - loss: 0.8989 - accuracy: 0.5849
Epoch 23/30
21/21 [==============================] - 30s 1s/step - loss: 0.9571 - accuracy: 0.5185
Epoch 24/30
21/21 [==============================] - 30s 1s/step - loss: 0.8997 - accuracy: 0.5617
Epoch 25/30
21/21 [==============================] - 29s 1s/step - loss: 0.8274 - accuracy: 0.6080
Epoch 26/30
21/21 [==============================] - 30s 1s/step - loss: 0.8021 - accuracy: 0.6404
Epoch 27/30
21/21 [==============================] - 29s 1s/step - loss: 0.8043 - accuracy: 0.6188
Epoch 28/30
21/21 [===

In [103]:
crossval_loso(threedConv, 100, X_loso, y_loso)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)
------------------------------------------------------------------------
Training for fold 0 ...
3/3 [==============================] - 0s 5ms/step - loss: 0.4774 - accuracy: 0.7361
Score for fold 0 - accuracy: 73.611110%
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)
------------------------------------------------------------------------
Training for fold 1 ...
3/3 [==============================] - 0s 5ms/step - loss: 0.4765 - accuracy: 0.7500
Score for fold 1 - accuracy: 75.000000%
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)
------------------------------------------------------------------------
Training for fold 2 ...
3/3 [==============================] - 0s 5ms/step - loss: 0.4720 - accuracy: 0.7361
Score for fold 2 - accuracy: 73.611110%
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11

In [105]:
crossval_loso(convlstm2d, 30, X_loso, y_loso)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)
------------------------------------------------------------------------
Training for fold 0 ...
Epoch 1/30
32/32 [==============================] - 44s 1s/step - loss: 1.4889 - accuracy: 0.3284
Epoch 2/30
32/32 [==============================] - 44s 1s/step - loss: 1.2868 - accuracy: 0.3909
Epoch 3/30
32/32 [==============================] - 44s 1s/step - loss: 1.3091 - accuracy: 0.3829
Epoch 4/30
32/32 [==============================] - 44s 1s/step - loss: 1.2133 - accuracy: 0.4276
Epoch 5/30
32/32 [==============================] - 44s 1s/step - loss: 1.1022 - accuracy: 0.4534
Epoch 6/30
32/32 [==============================] - 44s 1s/step - loss: 1.0395 - accuracy: 0.4901
Epoch 7/30
32/32 [==============================] - 44s 1s/step - loss: 1.0546 - accuracy: 0.4732
Epoch 8/30
32/32 [==============================] - 45s 1s/step - loss: 0.9595 - accuracy: 0.5268
Epoch 9/30
32/32 [=

32/32 [==============================] - 45s 1s/step - loss: 0.7471 - accuracy: 0.6429
Epoch 17/30
32/32 [==============================] - 44s 1s/step - loss: 0.7205 - accuracy: 0.6637
Epoch 18/30
32/32 [==============================] - 44s 1s/step - loss: 0.7831 - accuracy: 0.6200
Epoch 19/30
32/32 [==============================] - 44s 1s/step - loss: 0.7307 - accuracy: 0.6349
Epoch 20/30
32/32 [==============================] - 45s 1s/step - loss: 0.7031 - accuracy: 0.6429
Epoch 21/30
32/32 [==============================] - 44s 1s/step - loss: 0.7191 - accuracy: 0.6389
Epoch 22/30
32/32 [==============================] - 45s 1s/step - loss: 0.6949 - accuracy: 0.6706
Epoch 23/30
32/32 [==============================] - 44s 1s/step - loss: 0.7466 - accuracy: 0.6250
Epoch 24/30
32/32 [==============================] - 44s 1s/step - loss: 0.6554 - accuracy: 0.6756
Epoch 25/30
32/32 [==============================] - 44s 1s/step - loss: 0.6602 - accuracy: 0.6577
Epoch 26/30
32/32 [===

32/32 [==============================] - 46s 1s/step - loss: 1.5144 - accuracy: 0.3641
Epoch 2/30
32/32 [==============================] - 46s 1s/step - loss: 1.1407 - accuracy: 0.4673
Epoch 3/30
32/32 [==============================] - 46s 1s/step - loss: 1.0933 - accuracy: 0.4812
Epoch 4/30
32/32 [==============================] - 45s 1s/step - loss: 1.0247 - accuracy: 0.5228
Epoch 5/30
32/32 [==============================] - 44s 1s/step - loss: 1.0049 - accuracy: 0.5089
Epoch 6/30
32/32 [==============================] - 43s 1s/step - loss: 0.9721 - accuracy: 0.5208
Epoch 7/30
32/32 [==============================] - 44s 1s/step - loss: 0.9366 - accuracy: 0.5615
Epoch 8/30
32/32 [==============================] - 44s 1s/step - loss: 0.9187 - accuracy: 0.5565
Epoch 9/30
32/32 [==============================] - 44s 1s/step - loss: 0.9657 - accuracy: 0.5407
Epoch 10/30
32/32 [==============================] - 43s 1s/step - loss: 1.0248 - accuracy: 0.4812
Epoch 11/30
32/32 [===========

32/32 [==============================] - 44s 1s/step - loss: 0.8322 - accuracy: 0.5893
Epoch 19/30
32/32 [==============================] - 44s 1s/step - loss: 0.7100 - accuracy: 0.6558
Epoch 20/30
32/32 [==============================] - 44s 1s/step - loss: 0.7336 - accuracy: 0.6270
Epoch 21/30
32/32 [==============================] - 44s 1s/step - loss: 0.6722 - accuracy: 0.6647
Epoch 22/30
32/32 [==============================] - 44s 1s/step - loss: 0.6615 - accuracy: 0.6706
Epoch 23/30
32/32 [==============================] - 44s 1s/step - loss: 0.7218 - accuracy: 0.6488
Epoch 24/30
32/32 [==============================] - 44s 1s/step - loss: 0.7468 - accuracy: 0.6260
Epoch 25/30
32/32 [==============================] - 45s 1s/step - loss: 0.7369 - accuracy: 0.6349
Epoch 26/30
32/32 [==============================] - 44s 1s/step - loss: 0.6304 - accuracy: 0.6875
Epoch 27/30
32/32 [==============================] - 44s 1s/step - loss: 0.6209 - accuracy: 0.6667
Epoch 28/30
32/32 [===

32/32 [==============================] - 43s 1s/step - loss: 1.1977 - accuracy: 0.4187
Epoch 3/30
32/32 [==============================] - 44s 1s/step - loss: 1.0552 - accuracy: 0.4950
Epoch 4/30
32/32 [==============================] - 43s 1s/step - loss: 1.0588 - accuracy: 0.4772
Epoch 5/30
32/32 [==============================] - 44s 1s/step - loss: 1.0057 - accuracy: 0.5308
Epoch 6/30
32/32 [==============================] - 44s 1s/step - loss: 0.9663 - accuracy: 0.5347
Epoch 7/30
32/32 [==============================] - 44s 1s/step - loss: 0.9863 - accuracy: 0.4931
Epoch 8/30
32/32 [==============================] - 44s 1s/step - loss: 0.9501 - accuracy: 0.5129
Epoch 9/30
32/32 [==============================] - 45s 1s/step - loss: 0.9451 - accuracy: 0.5536
Epoch 10/30
32/32 [==============================] - 44s 1s/step - loss: 0.9343 - accuracy: 0.5516
Epoch 11/30
32/32 [==============================] - 44s 1s/step - loss: 0.8714 - accuracy: 0.5863
Epoch 12/30
32/32 [==========

32/32 [==============================] - 44s 1s/step - loss: 0.7932 - accuracy: 0.6210
Epoch 20/30
32/32 [==============================] - 43s 1s/step - loss: 0.7481 - accuracy: 0.6329
Epoch 21/30
32/32 [==============================] - 44s 1s/step - loss: 0.7217 - accuracy: 0.6448
Epoch 22/30
32/32 [==============================] - 44s 1s/step - loss: 0.7489 - accuracy: 0.6438
Epoch 23/30
32/32 [==============================] - 44s 1s/step - loss: 0.7006 - accuracy: 0.6458
Epoch 24/30
32/32 [==============================] - 43s 1s/step - loss: 0.6803 - accuracy: 0.6627
Epoch 25/30
32/32 [==============================] - 44s 1s/step - loss: 0.6592 - accuracy: 0.6528
Epoch 26/30
32/32 [==============================] - 44s 1s/step - loss: 0.6363 - accuracy: 0.6895
Epoch 27/30
32/32 [==============================] - 43s 1s/step - loss: 0.6506 - accuracy: 0.6667
Epoch 28/30
32/32 [==============================] - 44s 1s/step - loss: 0.6710 - accuracy: 0.6607
Epoch 29/30
32/32 [===